SQLAlchemy


In [549]:
from sqlalchemy import Integer, String, Column, ForeignKey, create_engine, inspect, select, LargeBinary, Float
from sqlalchemy.orm import Session, declarative_base, relationship
from faker import Faker
import random

In [550]:
Base = declarative_base()

In [551]:
# Criando minhas tabelas, utilizando SQLAlchemy

class Client(Base):
    __tablename__ = 'bank_clients'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    cpf = Column(String(9))
    address = Column(String(9))

    account = relationship('Account', back_populates='user',
                           cascade='all, delete-orphan')

    def __repr__(self):
        return f'Client(id={self.id}, name={self.name}, cpf={self.cpf}, address={self.address})'


class Account(Base):
    __tablename__ = 'bank_accounts'

    id = Column(Integer, primary_key=True, autoincrement=True)
    type = Column(String)
    agency = Column(String)
    num = Column(Integer)
    client_id = Column(Integer, ForeignKey('bank_clients.id'), nullable=False)
    balance = Column(Float)

    user = relationship('Client', back_populates='account')

    def __repr__(self):
        return f'Account(id={self.id}, type={self.type}, agency={self.agency}, client_id={self.client_id}, balance={self.balance})'

In [552]:
engine = create_engine('sqlite://')
Base.metadata.create_all(engine)

In [553]:
# Checando as tabelas do sqlite

isp = inspect(engine)
print(isp.get_table_names())

['bank_accounts', 'bank_clients']


In [554]:
fake = Faker()

# Criando dados de maneira aleatória, para alimentar o banco de dados, utilizando Faker para os nomes e random para os números
with Session(engine) as session:
    accounts = []
    accounts_type = ['CC', 'CS']
    for i in range(1, 101):
        num = 0
        accounts.append(Account(type=accounts_type[random.randint(0, 1)], agency='0001', num=i, balance=random.randint(0, 10000),
                        user=Client(name=fake.name(), cpf=str(random.randint(100_000_000, 999_999_999)), address=str(random.randint(100_000_000, 999_999_999)))))

    session.add_all(accounts)
    session.commit()

In [555]:
conn = engine.connect()

# Realizando consulta utilizando join
stmt_join = select("*").join_from(Client, Account)
results = conn.execute(stmt_join).fetchall()

data = []
for i in results:
    data.append(list(i))

In [556]:
import pandas as pd

# Criando DataFrame com os dados vindo do SQLite
df_balance = pd.DataFrame(data, columns=[
                          'id_client', 'client', 'cpf', 'cep', 'account_id', 'acc_type', 'agency', 'num', 'client_id', 'balance'])
df = df_balance[['id_client', 'client', 'cpf',
                 'cep', 'acc_type', 'agency', 'num', 'balance']]

In [557]:
df.head()

,id_client,client,cpf,cep,acc_type,agency,num,balance
0,1,Cathy Bautista,150489040,322332609,CC,0001,1,1279.0
1,2,Jacob Smith,601982794,439811173,CC,0001,2,2184.0
2,3,Jared Campbell,437239788,668129893,CS,0001,3,2639.0
3,4,Ana Becker,258336363,134707812,CS,0001,4,1528.0
4,5,Morgan Watson,250232242,849965633,CS,0001,5,2042.0


Pymongo


In [558]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = ''

client = MongoClient(uri, server_api=ServerApi('1'))

In [559]:
# Conectando na collection

db = client.meubanco
meubanco = db.bank

# Loop com os dados fakes criados para alimentar o banco de dados SQL
for i in data:
    account = {
        'type': i[5],
        'agency': i[6],
        'num': i[7],
        'id_client': i[0],
        'balance': i[9],
        'client': {
            'id': i[0],
            'name': i[1],
            'cpf': i[2],
            'address': i[3]
        }
    }
    meubanco.insert_one(account)

In [561]:
# Realizar um print em todos os clientes com mias de 9000 na conta
for r in meubanco.find({'balance': {'$gt': 9000}}):
    print(r)

{'_id': ObjectId('65ad9b1cd0b20ffddd687d4d'), 'type': 'CS', 'agency': '0001', 'num': 14, 'id_client': 14, 'balance': 9235.0, 'client': {'id': 14, 'name': 'Julia Andrews', 'cpf': '397864443', 'address': '836335875'}}
{'_id': ObjectId('65ad9b1dd0b20ffddd687d53'), 'type': 'CC', 'agency': '0001', 'num': 20, 'id_client': 20, 'balance': 9779.0, 'client': {'id': 20, 'name': 'Julie Turner', 'cpf': '821571221', 'address': '471774216'}}
{'_id': ObjectId('65ad9b1dd0b20ffddd687d62'), 'type': 'CC', 'agency': '0001', 'num': 35, 'id_client': 35, 'balance': 9744.0, 'client': {'id': 35, 'name': 'Michele Smith', 'cpf': '542275174', 'address': '637075996'}}
{'_id': ObjectId('65ad9b1dd0b20ffddd687d63'), 'type': 'CS', 'agency': '0001', 'num': 36, 'id_client': 36, 'balance': 9617.0, 'client': {'id': 36, 'name': 'Nicole Compton', 'cpf': '269555265', 'address': '501142578'}}
{'_id': ObjectId('65ad9b1dd0b20ffddd687d64'), 'type': 'CS', 'agency': '0001', 'num': 37, 'id_client': 37, 'balance': 9623.0, 'client': {

In [562]:
# Todos os registros
for account in meubanco.find():
    pprint.pprint(account)

{'_id': ObjectId('65ad9b1cd0b20ffddd687d40'),
 'agency': '0001',
 'balance': 1114.0,
 'client': {'address': '300740199',
            'cpf': '199868507',
            'id': 1,
            'name': 'Emily Moore'},
 'id_client': 1,
 'num': 1,
 'type': 'CS'}
{'_id': ObjectId('65ad9b1cd0b20ffddd687d41'),
 'agency': '0001',
 'balance': 2104.0,
 'client': {'address': '258259829',
            'cpf': '845046290',
            'id': 2,
            'name': 'William Davidson'},
 'id_client': 2,
 'num': 2,
 'type': 'CC'}
{'_id': ObjectId('65ad9b1cd0b20ffddd687d42'),
 'agency': '0001',
 'balance': 1534.0,
 'client': {'address': '562996745',
            'cpf': '830682787',
            'id': 3,
            'name': 'Breanna Jones'},
 'id_client': 3,
 'num': 3,
 'type': 'CS'}
{'_id': ObjectId('65ad9b1cd0b20ffddd687d43'),
 'agency': '0001',
 'balance': 4010.0,
 'client': {'address': '765412392',
            'cpf': '656026611',
            'id': 4,
            'name': 'Madison Simmons DVM'},
 'id_client': 4,